# Балансировка классов

In [ ]:
import lightgbm as lgb
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
categoryFeatures = ['!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!']
params = {
    'boosting_type': 'GBDT',
    'objective':"binary",
    'metric':'binary_logloss',
    'random_state': 42,
}

for i in range(20):
	positive_count_train = y.value_counts()[1]
	sampler = RandomUnderSampler(sampling_strategy={0: positive_count_train, 1: positive_count_train},random_state=i, replacement=True)
	X_re, y_re = sampler.fit_resample(X, y)
	
	lgb_train = lgb.Dataset(X_re.drop("Id",axis=1), y_re,free_raw_data=False)
	model = lgb.train(params, lgb_train, 
					  categorical_feature = categoryFeatures,
					  num_boost_round=500)
	pred = model.predict(X_test.drop("Id",axis=1),num_iteration=model.best_iteration)
	if i == 0:
		output2 = pd.DataFrame(pred, columns=['pred' + str(i + 1)])
	else:
		output = pd.DataFrame(pred, columns=['pred' + str(i + 1)])
		output2 = pd.concat([output2, output], axis=1)


Вариант балансировка классов когда мы берем нулевой класс рандомно, и по количеству сопоставимо с первым классом

In [ ]:
categorical_features = list(df_lc.select_dtypes(include=['object']).columns)
numeric_features = list(set(df_lc.columns) - (set(categorical_features) | set(['target'])))
all_features = categorical_features + numeric_features

# Разделение на тренировочный и тестовый наборы данных
train, test = train_test_split(df_lc, test_size=0.2, random_state=42)

# Отбор данных класса 1
class_1 = train[train['target'] == 1]
# Отбор данных класса 0
class_0 = train[train['target'] == 0].sample(n=len(class_1)*2, random_state=42)
# Объединение выборок
balanced_train = pd.concat([class_1, class_0])

X_train_b = balanced_train[all_features]
y_train_b = balanced_train['target']

X_test_b = test[all_features]
y_test_b = test['target']


# Визуализация

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

def plot_histogram(data, column, bins=30, kde=True, color='blue'):
    
    plt.figure(figsize=(15, 5))
    sns.histplot(data[column], bins=bins, kde=kde, color=color, log_scale = True)
    plt.title(f'Histogram and KDE of {column}') #Kernel Density Estimation (ядерное сглаживание)
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.xlim([1e-6,1])
    plt.ylim([0,4e5])
    plt.show()

def plot_boxplot(data, column):
    
    plt.figure(figsize=(8, 6))
    sns.boxplot(y=data[column])
    plt.title(f'Boxplot of {column}')
    plt.ylabel(column)
    plt.grid(True)
    plt.show()

def plot_scatter(data, x_col, y_col):

    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=data, x=x_col, y=y_col)
    plt.title(f'Scatter Plot between {x_col} and {y_col}')
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    plt.grid(True)
    plt.show()

def plot_qq(data, column):

    plt.figure(figsize=(8, 6))
    stats.probplot(data[column], dist="norm", plot=plt)
    plt.title(f'Q-Q plot of {column}')
    plt.grid(True)
    plt.show()


In [ ]:
correlation_matrix = df.corr()
# Создаем тепловую карту
plt.figure(figsize=(10, 8))  
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5, cbar=True)
plt.title('Тепловая карта корреляций')
plt.show()


In [ ]:
rides_info.drop("sex", axis=1).hist(figsize=(20, 15), layout=(-1, 5));

In [ ]:
import seaborn as sns
sns.set_theme()  # Тут можно задать стили

g = sns.relplot(
    data=rides_info,
    x="ride_date",
    y="deviation_normal",
    hue="target_class",
    kind="line",  # или scatter
    aspect=4,
)
g.set_xticklabels(rotation=45, horizontalalignment="right", step=2);

- Замечаем точки перегиба
- Замечаем точки входа
- Возможно углы наклона до перегиба

In [ ]:
Когорты
Cog = df.pivot_table(index = 'столбец месяц первой покупки',columns = 'столбец индекс',values = 'суммируемая величина (ID)', aggfunc = len)
aggfunc  len - кол-во sum -сумма pd.Series.nunique - кол-во уникальных

In [ ]:
https://github.com/a-milenkin/Competitive_Data_Science/blob/main/notebooks/2.3%20-%20Visualisation.ipynb

# Заполнение пропусков

In [ ]:
# Simple
numerical = df.select_dtypes(include=['int64', 'float64']).columns

for feat in numerical:
    df[feat].fillna(df[feat].median(), inplace=True)
df.isnull().sum() # проверка

In [ ]:
# Knn
# ищет ближайших соседей и усредняет значения колонки по ближайшим соседям 
from sklearn.impute import KNNImputer
def fill_empty(df):
    featuresWithNulls = df.columns[df.isnull().any()]
    imputer = KNNImputer(n_neighbors=5) 
    df[featuresWithNulls] = imputer.fit_transform(df[featuresWithNulls])
    return df


In [ ]:
# Knn
# ищет ближайших соседей и усредняет значения колонки по ближайшим соседям 
from sklearn.impute import KNNImputer
def fill_empty(df):
    # Стандартизация признаков
    scaler = StandardScaler()
    scaled_df = scaler.fit_transform(df)
    scaled_df = pd.DataFrame(scaled_df, columns=df.columns)
    
    # Определение признаков по которым происходит поиск соседей
    featuresWithNulls = scaled_df.columns[scaled_df.isnull().any()]
    # Применение KNNImputer
    imputer = KNNImputer(n_neighbors=5)
    scaled_df[featuresWithNulls] = imputer.fit_transform(scaled_df[featuresWithNulls])
    
    # Возвращение данных к исходному масштабу
    return pd.DataFrame(scaler.inverse_transform(scaled_df), columns=df.columns)

In [ ]:
#Knn
# ищет ближайших соседей и усредняет значения колонки по ближайшим соседям 
from sklearn.impute import KNNImputer
from sklearn.neighbors import NearestNeighbors

columns_for_imputation = ['feature1', 'feature2']
nn = NearestNeighbors(n_neighbors=5)
nn.fit(df[columns_for_imputation])

# Для примера выводим расстояния для первой строки
distances, indices = nn.kneighbors([df.iloc[0][columns_for_imputation].to_numpy()])
print("Расстояния:", distances)
print("Индексы ближайших соседей:", indices)

In [ ]:
categorical = df.select_dtypes(include=['object']).columns
for feat in categorical:
    df[feat].fillna(df[feat].mode()[0], inplace=True)

In [ ]:
# Интерполяция
Метод .interpolate() Имеет возможности линейной, полиномиальной, экспоненциальной, временной и пр. интерполяции.

# Оценка выбросов

In [ ]:
def calculate_outliers_3sigma(df, col):
    mat=df[col].mean()
    sigma = (((df[col] - mat)**2).sum()/len(df[col]))**(1/2)
    interval = (mat-3*sigma,mat+3*sigma)
    
    #формируем границы интервала (по сути это фильтр)
    is_outlier = (df[col] < interval[0]) | (df[col] > interval[1])
   
    #процент вышедших за пределы интервала
    outlier = pd.DataFrame()
    outlier['col'] = np.array([col])
    proc = round(is_outlier.sum()/len(df)*100,0)
    outlier['%'] = np.array([proc])
    outlier['-3Sig'] = np.array([round(interval[0],2)])
    outlier['+3Sig'] = np.array([round(interval[1],2)])
           
    lst_out= list(is_outlier[is_outlier==True].index)
    outlier['ind'] = 1
    outlier['ind'] = outlier['ind'].astype('object')
    outlier.at[0,'ind'] = lst_out
    #возвращает фильтр на строки где есть выбросы
    return outlier, is_outlier 

# USES

colum = df_faiss.select_dtypes('number').columns
out_of_data = pd.DataFrame()
for j in colum:
    outlier, is_outlier = calculate_outliers_3sigma(df,j) # изменить DF
    out_of_data = pd.concat([out_of_data,outlier])
out_of_data.set_index('col', inplace=True) 

# Проверка выбросов
check_indx = out_of_data.loc[признак,'ind']
df[df.index.isin(check_indx)].признак

In [ ]:
def calculate_outliers_percentile(df, col, lower_percentile=1, upper_percentile=99):
       # Вычисление перцентильных значений
    lower_bound = np.percentile(df[col].dropna(), lower_percentile)
    upper_bound = np.percentile(df[col].dropna(), upper_percentile)
    
    # Формирование фильтра выбросов
    is_outlier = (df[col] < lower_bound) | (df[col] > upper_bound)
    
    # Подсчёт процента выбросов
    outlier = pd.DataFrame()
    outlier['col'] = np.array([col])
    percent_outliers = round(is_outlier.mean() * 100, 2)
    outlier['%'] = np.array([percent_outliers])
    outlier['LowerBound'] = np.array([round(lower_bound, 2)])
    outlier['UpperBound'] = np.array([round(upper_bound, 2)])
    
    # Сохранение индексов выбросов
    list_outliers = list(is_outlier[is_outlier].index)
    outlier['ind'] = 1
    outlier['ind'] = outlier['ind'].astype('object')
    outlier.at[0, 'ind'] = list_outliers
    
    # Возвращает статистику по выбросам и фильтр
    return outlier, is_outlier

# USES
colum = df_faiss.select_dtypes('number').columns
out_of_data = pd.DataFrame()
for j in colum:
    outlier, is_outlier = calculate_outliers_percentile(df, j) # изменить DF
    out_of_data = pd.concat([out_of_data,outlier])
out_of_data.set_index('col', inplace=True)


# Энкодеры

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
for column in ['Fruit', 'Color']:
    df[column + '_encoded'] = encoder.fit_transform(df[column])

In [ ]:

from category_encoders import BinaryEncoder

encoder = BinaryEncoder(cols=['Fruit'], drop_invariant=True)  # 'cols' указывает 
df_encoded = encoder.fit_transform(df)

In [ ]:
freq = df[признак].value_counts() / df.shape[0]
df[признак_новый] = df[признак].map(freq)

In [ ]:
def target_encode(df, categorical_col, target_col, smoothing=1):
    # Вычисляем глобальное среднее значение целевой переменной
    mean = df[target_col].mean()
    # Агрегация среднего по категориям
    agg = df.groupby(categorical_col)[target_col].agg(['count', 'mean'])
    # Вычисляем сглаженные средние
    counts = agg['count']
    means = agg['mean']
    smooth = (counts * means + smoothing * mean) / (counts + smoothing)
    
    return df[categorical_col].map(smooth)


data['category_encoded'] = target_encode(data, 'category', 'target', smoothing=10)

In [ ]:
# Бинарное дерево
class Node:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None


    def insert(self, node):

        if self.key > node.key:
            if self.left is None:
                self.left = node
            else:
                self.left.insert(node)

        elif self.key < node.key:
            if self.right is None:
                self.right = node
            else:
                self.right.insert(node)

# Использование класса Node и метода insert
root = Node(10)
root.insert(Node(5))
root.insert(Node(15))
root.insert(Node(3)) 
root.insert(Node(7))

# Cat

In [ ]:
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score, recall_score, f1_score

categorical_features = list(df.select_dtypes(include=['object']).columns)
numeric_features = list(set(df.columns) - (set(categorical_features) | set(['is_promoted','employee_id'])))

X = df[categorical_features + numeric_features]
y = df['is_promoted']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = CatBoostClassifier(
    iterations=200,
    learning_rate=0.1,
    depth=3,
    cat_features=categorical_features,
    verbose=10,
    #auto_class_weights='Balanced'
)
model.fit(X_train, y_train, sample_weight=weights)

# Предсказание и оценка 
threshold =0.7
y_proba = model.predict_proba(X_test)[:, 1]  
y_pred = (y_proba>=threshold).astype(int)


# вероятности для класса 1 # Расчет метрик 
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC AUC: {roc_auc:.2f}')

# Вычисление Precision, Recall и F1 Score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)


print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')


cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(include_values=True, cmap='coolwarm', ax=None, xticks_rotation='horizontal')

plt.show()

In [ ]:
    # Получение и сохранение результатов на валидационном наборе
    predictions = model_mango_lc1.predict_proba(X_valid_fold)[:, 1]
    fold_result = {
        'fold_number': fold_number,
        'validation_AUC': model_mango_lc1.best_score_['validation']['AUC'],
        'best_iteration': model_mango_lc1.get_best_iteration()}
                      
    fold_results.append(fold_result)


    print(f"Результаты для фолда {fold_number}:")
    print(f"Лучшая итерация: {model_mango_lc1.get_best_iteration()}")
    print(f"Точность на валидации: {model_mango_lc1.best_score_['validation']['AUC']:.2f}")

print('Итоги кросс-валидации:')
for result in fold_results:
    print(f"Фолд {result['fold_number']}: Лучшая итерация - {result['best_iteration']}, Accuracy - {result['validation_AUC']:.2f}")
```
Eval_metric может быть даже кастомный. Для использования нужно ее указать как eval_metric = custom_metric_object
```Python
from catboost import metrics

def balanced_log_loss(y_true, y_pred):
    # Ваши вычисления здесь, как в вашем примере выше
    N0, N1 = np.bincount(y_true)
    
    y0 = np.where(y_true == 0, 1, 0)
    y1 = np.where(y_true == 1, 1, 0)
    
    eps = 1e-15
    y_pred = np.clip(y_pred, eps, 1 - eps)
    p0 = np.log(1 - y_pred)
    p1 = np.log(y_pred)
    return -(1 / N0 * np.sum(y0 * p0) + 1 / N1 * np.sum(y1 * p1)) * 0.5

# Создание объекта метрики для CatBoost
custom_metric_object = metrics.make_single_metric('balanced_log_loss', balanced_log_loss, is_max_optimal=False)

# Рекурсивный отбор

In [ ]:
def feature_selection_by_roc_auc(df, features, target = 'target'):
    X = df[features]
    y = df[target]
    removed_features = []
    roc_aucs = []
    current_roc_auc = 0.9


    for feature in features:
        # Тренируем модель без одного признака
        X_train, X_test, y_train, y_test = train_test_split(X.drop(feature, axis=1), y, test_size=0.2, random_state=42)
        categorical_features = list(X.drop(feature, axis=1).select_dtypes(include=['object']).columns)
        encoder = TargetEncoder(cols = categorical_features)
        X_train_encoded = encoder.fit_transform(X_train, y_train)
        X_test_encoded = encoder.transform(X_test)
        
        model = LGBMClassifier(is_unbalance = True, random_state=42)
        model.fit(X_train_encoded, y_train)
        
        y_pred_prob = model.predict_proba(X_test_encoded)[:, 1]
        new_roc_auc = roc_auc_score(y_test, y_pred_prob)
        print(f"{feature} : {new_roc_auc}")
        roc_aucs.append(new_roc_auc)


        # Проверяем, улучшился ли ROC AUC достаточно, чтобы исключить признак
        if new_roc_auc > current_roc_auc + 0.005:
            removed_features.append(feature)
            current_roc_auc = new_roc_auc
            X.drop(feature, axis=1, inplace=True)  # Удаляем признак навсегда
        else:
            # Если не улучшилось достаточно, возвращаем признак обратно
            continue


    return removed_features, roc_aucs



removed_features, roc_aucs = feature_selection_by_roc_auc(df, all_features)
print("Removed features:", removed_features)
print("ROC AUCs for each step:", roc_aucs)

# LGBM

## Simple

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from functools import partial
import numpy as np
import lightgbm as lgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from category_encoders import TargetEncoder

X = df_lc[all_features]
y = df_lc['target']

categorical_features = list(df_lc.select_dtypes(include=['object']).columns)
numeric_features = list(set(df_lc.columns) - (set(categorical_features) | set(['target'])))
all_features = categorical_features + numeric_features

X_tmp, X_test, y_tmp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)
X_train, X_valid, y_train, y_valid = train_test_split(X_tmp, y_tmp, test_size=0.2, random_state=42, stratify = y_tmp)

encoder = TargetEncoder(cols = categorical_features) # ЭНКОДЕ НА ТРЕНИРОВКЕ И НА ОБУЧЕНИИ ЭТО ОДИН ЭНКОДЕР
X_train_encoded = encoder.fit_transform(X_train, y_train)
X_test_encoded = encoder.transform(X_test)
X_valid_encoded = encoder.transform(X_valid)

lgb_params = {
    'is_unbalance': True,
    #'learning_rate': 0.1,
    #'max_depth': 50,
    #'n_estimators': 1000,
    #'num_leaves': 31,
    #'min_child_samples': 20,
    #'min_split_gain': 0,
    #'subsample': 0.8,
    #'subsample_freq': 5,
    #'colsample_bytree': 0.8,
    #'reg_alpha': 0,
    #'objective': 'binary',
    #'reg_lambda': 1,
    #'scale_pos_weight': 1,
    #'n_jobs': 15,
    'verbose': 10

model = lgb.LGBMClassifier(**lgb_params)
model.fit(
    X_train_encoded, 
    y_train, 
    eval_set = [(X_valid_encoded, y_valid)], 
    eval_metric = 'binary_loss', 
    early_stopping_rounds = 30, 
    verbose = True)
}

# Предсказание и оценка 
threshold =0.9
y_proba = model.predict_proba(X_test_encoded)[:, 1]  
y_pred = (y_proba>=threshold).astype(int)


# вероятности для класса 1 # Расчет метрик 
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC AUC: {roc_auc:.2f}')

# Вычисление Precision, Recall и F1 Score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)


print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')

plt.show()

## LGBM + hyperopt

In [ ]:
# Пространство поиска
param_space = {
    'learning_rate': hp.choice('learning_rate', [0.1]),  # только одно значение для упрощения
    'n_estimators' : hp.choice('n_estimators', [500]),
    'min_child_samples' : scope.int(hp.quniform('min_child_samples', 10, 200, 1)),
    'min_split_gain' : hp.uniform('min_split_gain', 0, 1),
    'subsample' : hp.uniform('subsample', 0.1, 1),
    'subsample_freq' : scope.int(hp.quniform('subsample_freq', 1, 10, 1)),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1, 0.1),
    'reg_alpha' : scope.int(hp.quniform('reg_alpha', 1, 10, 1)),
    'reg_lambda' : scope.int(hp.quniform('reg_lambda', 1, 10, 1)),
    'max_depth': scope.int(hp.quniform('max_depth', 5, 50, 1)), 
    'scale_pos_weight': scope.int(hp.quniform('scale_pos_weight', 1, 30, 1)),
    'num_leaves': scope.int(hp.quniform('num_leaves', 2, 200, 1))       
}


# Функция оптимизации с данными
optimization_function = partial(optimize, X_train= X_train_encoded, X_valid = X_valid_encoded, y_train= y_train, y_valid= y_valid)


# Запуск оптимизации
trials = Trials()
best_params = fmin(optimization_function, space=param_space, algo=tpe.suggest, max_evals=50, trials=trials)


print("Лучшие параметры:")
print(space_eval(param_space, best_params))

## LGBM + Strarified + Kfold + hyperopt

In [ ]:
    # Создаем объект StratifiedKFold
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    auc_scores = []


    # Создаем таргет энкодер для категориальных переменных
    if categorical_features:
        encoder = TargetEncoder(cols=categorical_features)


    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Применяем таргет энкодер, если есть категориальные переменные
        if categorical_features:
            X_train[categorical_features] = encoder.fit_transform(X_train[categorical_features], y_train)
            X_test[categorical_features] = encoder.transform(X_test[categorical_features])
        
        # Обучаем модель на текущем фолде
        model = lgb.LGBMClassifier(**params, n_jobs=-1)
        model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=False)
        
        # Оцениваем модель на тестовом фолде
        preds = model.predict_proba(X_test)[:, 1]
        auc_score = roc_auc_score(y_test, preds)
        auc_scores.append(auc_score)
    
    # Возвращаем средний AUC по всем фолдам
    mean_auc = np.mean(auc_scores)
    return {'loss': -mean_auc, 'status': STATUS_OK, 'model': model}


# Пространство поиска
param_space = {
    'learning_rate': hp.choice('learning_rate', [0.1]),  # только одно значение для упрощения
    'max_depth': scope.int(hp.quniform('max_depth', 5, 50, 1)),           
    'num_leaves': scope.int(hp.quniform('num_leaves', 2, 200, 1))       
}

# Функция оптимизации с данными
optimization_function = partial(optimize, X=X, y=y, n_folds=3, categorical_features=categorical_features)


# Запуск оптимизации
trials = Trials()
best_params = fmin(optimization_function, space=param_space, algo=tpe.suggest, max_evals=50, trials=trials)


print("Лучшие параметры:")
print(space_eval(param_space, best_params))

In [ ]:
new_params = space_eval(param_space, best_params)

tuned_model = lgb.LGBMClassifier(**new_params)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

X_train_encoded = encoder.fit_transform(X_train, y_train)
X_test_encoded = encoder.transform(X_test)

tuned_model.fit(X_train_encoded,y_train)

In [ ]:
# диапазон параметров для перебора
param_space = {
     'metric' : hp.choice('metric', ['logloss']),
    'learning_rate' : hp.choice('learning_rate', [0.01]),
    'max_depth' : scope.int(hp.quniform('max_depth', 5, 50, 1)),
    'n_estimators' : hp.choice('n_estimators', [500]),
    'num_leaves' : scope.int(hp.quniform('num_leaves', 2, 200, 1)),
    'min_child_samples' : scope.int(hp.quniform('min_child_samples', 10, 200, 1)),
    'min_split_gain' : hp.uniform('min_split_gain', 0, 1),
    'subsample' : hp.uniform('subsample', 0.1, 1),
    'subsample_freq' : scope.int(hp.quniform('subsample_freq', 1, 10, 1)),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1, 0.1),
    'reg_alpha' : scope.int(hp.quniform('reg_alpha', 1, 10, 1)),
    'reg_lambda' : scope.int(hp.quniform('reg_lambda', 1, 10, 1)),
    'scale_pos_weight': scope.int(hp.quniform('scale_pos_weight', 1, 30, 1)),
    'random_state' : hp.choice('random_state', [22]),
    'objective' : hp.choice('objective', ['binary']),
    'verbose' : hp.choice('verbose', [-1])
}

# Получение важности признаков
feature_imp = pd.DataFrame({'cols':X_train.columns,'importances':estimator_base.feature_importances_}).sort_values(by ='importances',ascending = False).head(15)
feature_imp

model.get_params()
model.feature_importances_
model.best_scores_

# Pandas

## Group By

In [ ]:
.agg(name = ('column_name','sum'))

['column_name'].mean()
	.shift(1) #сдвиг на 1
	.cumsum() #накопленная сумма
	.sample(frac=1/3, random_state=2) # набрать рандомно

## Roling (окно скользящего среднего)

In [ ]:
df.groupby('client_id')['sum']
.rolling(window =3,min_period = 1) # окно 3, минимально хоть 1 не Nan 
.mean().reset_index(drop=True)
```

## соединения таблиц

In [ ]:
# Строк одинаково, не преремешано
df_combined = pd.concat([df_1, df_2, df_3], axis=1) 

In [ ]:
# Есть колонка индекса и они перемешаны
df_1.set_index('index_col', inplace=True)
df_2.set_index('index_col', inplace=True)
df_3.set_index('index_col', inplace=True)


df_combined = pd.concat([df_1, df_2, df_3], axis=1)
df_combined.reset_index(inplace=True)

In [ ]:
pd.merge(left =data1 , right = data2, on='id', how ='inner') # по id
pd.merge(df1, df2, left_index=True, right_index=True) # по индексу

## со строками 

In [ ]:
df.признак.str.lower() / .str.upper()
df.признак.str.upper()
df.признак.str.len()
df.признак.str.strip() # убрать пробелы в начале и в конце
df.признак.str[3] # обратиться внутри Series к 3-ему элементу
df.признак.str.cat(sep='_') # собрать всю колонку признаков в одну строчку
df.признак.str.startswith('G') # True False начинается с ‘G’
df.признак.str.endswith('1') # True False заканчивается на ‘G’
df.признак.str.replace('Geeks', 'Gulshan') # **замена элемента
df.признак.str.count('n') # подсчет кол-ва вхождения эл-ов
df.признак.str.find('n') # возвращает индекс найденного
df.признак.str.findall('n') # возвращает список вхождения
df.признак.str.replace('price','old_price') # переименование колонки
df.признак.str.contains('чай') # содержит символы

In [ ]:
## Даты

from datetime import date
df[признак] = pd.to_datetime(df.признак) # object →  datetime64
df[признак]= pd.to_datetime(df.признак,unit ='s') # 5345648 → datetime64
df.признак.dt.year # отдельно год (на весь SERIES)
# quarter month week day hour minute dayofweek dayofyear
df.признак = df.date.dt.strftime('%Y-%m-%d, %H:%M:%S') # форматирование
# day (%a Sun) (%A Sunday) (%d 07)
# week (%W 01)
# month (%b Jan) (%B January) (%m 01)
# year (%Y J2022) (%y 22)
df[признак] = pd.to_datetime(df[['year', 'month', 'day']]) # сбор даты
pd.to_datetime(ratings.timestamp, unit='s') # из вот такого формата '1260759144'
date.today().strftime("%Y-%m-%d")

In [ ]:
# вычисляет разницу сегодня между дата, чтобы округлить в годах, по строкам где есть NAN
((pd.to_datetime('today') - df[признак_дата]) / np.timedelta64(1, 'Y')).astype(float)

## Чтение и сохранение

In [ ]:
# Запись
df.to_csv('./название.csv',sep = '\t',index=False)
df.to_excel('./название.xlsx',sheet_name='<лист>' , index_label='id' )
df.to_json('/content/drive/My Drive/название.json', orient='records', force_ascii=False)
df.to_pickle ("./content/название.pkl")

# Чтение
pd.read_csv( './название.csv', sep= ';', index_col= <столбец>)
pd.read_excel( './название.xlsx', sheet_name= '<лист>', index_col= <столбец>,dtype ={0:str,1:str})
pd.read_json( './название.xlsx', inplace=True)
pd.read_pickle ('./название.pkl')

# Разное
# из CSV в Numpy
preds_pers = np.genfromtxt('./название.csv', delimiter=',', skip_header=1)

## Замена

In [ ]:
# Simple filter
df.loc[:,'y'] = 0.1
df.loc[df["y"] > 2, "y"] = -1
df.loc[df.y.isna(),'y'] = 0.1

# WHERE
df['new_price'] = np.where(df['price']>100,'expensive','cheap')

# MAP
mapping_dict = {'red':1, 'blue':2, 'yellow':3}
df['n_color'] = df['color'].map(mapping_dict)

# APPLY
df['new_price'] = df.apply(lambda x: x['price']*1.05 if x['price']>100 and x['name']!='икра' else x['price'], axis =1)

#CONDITIONS
conditions = [ 
(df['Age'] < 20), 
(df['Age'] >= 20) & (df['Age'] < 30), 
(df['Age'] >= 30) & (df['Age'] < 40), 
(df['Age'] >= 40) & (df['Age'] < 50), 
(df['Age'] >= 50) ] 

choices = [ 'Teenager', 'Twenties', 'Thirties', 'Forties', 'Fifty and older' ]
df['Category'] = np.select(conditions, choices, default='Not Specified')

## Индексы и переименование 

In [ ]:
df.set_index(['id', 'item']) # сделать новые индексы из колонок**
df.rename(columns={"old_name" : "new_name", "old_name" : "new_name"}, inplace=True)
df.reset_index()

## Дубли

In [ ]:
df.duplicated().sum() # количество дублей
df[df.duplicated()==True] # найти дубли
df[df.duplicated(keep=False)] # показать оба дубля
df = df.drop_duplicates() # удалить дубли
df.drop_duplicates(subset=['object_id', 'object', 'type', 'lat', 'lon'], keep=False, inplace=True) #Убрать дубли по набору признаков (убрать оба дубля)
df_emb = df_emb.drop_duplicates(subset = 'Code', keep ='first')


#сравнить два датафрейма
merged = pd.merge(df1,df2, on = list_y, how = 'outer', indicator= True) #list_y - список признаков для сравнения
only_in_df1 = merged[merged['_merge']=='left_only']
only_in_df2 = merged[merged['_merge']=='right_only']

## Типы

In [ ]:
select_dtypes(include=['float64','object']).columns
df.название_колонки.describe().apply(lambda x: f"{x:0.3f}")

## Cross

In [ ]:
df.pivot_table(index = 'first_date_cog', columns = 'cogort', values = 'price', aggfunc = sum)
# aggfunc: len sum pd.Series.nunique

df.pivot_table(index = 'userId',columns = 'movieId', values = 'rating')

pd.crosstab(df['userId'], df['movieId'], values=df['rating'], aggfunc='first')

# Проверка Nan
pivot_df.isnull().all().sum()
pivot_df.columns[pivot_df.isnull().all()]
pivot_df.index[pivot_df.isnull().all(axis=1)]

# Удаление Nan
pivot_df = pivot_df.dropna(axis=1, how='all')
pivot_df = pivot_df.dropna(axis=0, how='all')
pivot_df.loc[1,:]

#МУЛЬТИИНДЕКС
user_ids = df.index.get_level_values('userId')  
movie_ids = df.index.get_level_values('movieId')  
specific_user = df.loc[(15.0, 2)]  
  
for (user_id, movie_id), data in df.iterrows():  
    print(f"User ID: {user_id}, Movie ID: {movie_id}")  
  
for userId, df in X_test.groupby('userId'):  
    a=df

In [ ]:
df['new_col]=df.groupby('user_id',as_index = False)['date'].transform('min')

функции трансформ:'min' 'max' 'sum' 'mean'  'nunique'
.transform( lambda x: sum([1 if i == 'cancel_order'  else 0 for i in x] ))

In [ ]:
## Связка `groupby` + `agg()` +  выдуманная функция

In [ ]:
# число уникальных значений
f_nuniq = lambda x: x.nunique()

# число значений, которые больше чем n
more_than_n_func = lambda x, n=8: sum(x > n)

# 30% перцентиль / квантиль уровня 0.3
def quant_func(x):
    return x.quantile(0.3)

# Функции поиска первой и второй моды для категориальных значений
first_mode = lambda x: x.value_counts().index[0]
second_mode = lambda x: x.value_counts().index[1]

fix_info_gr = fix_info.groupby("car_id", as_index=False).agg(
    
    # Все встроенные статистики
    worker_count=("worker_id", "count"),
    work_duration_mean=("work_duration", "mean"),
    work_duration_max=("work_duration", "max"),
    destroy_degree_std=("destroy_degree", "std"),
    destroy_degree_sum=("destroy_degree", "sum"),
    
    # Самописные функции для категорий
    work_type_nuniq=("work_type", f_nuniq),
    work_type_mode=("work_type", first_mode),
    work_type_second_mode=("work_type", second_mode),
    
    # Самописные функции для численных
    destroy_degree_crit_q=("destroy_degree", more_than_n_func),
    worker_quant_exp=("worker_experience", quant_func),
)

In [ ]:
## Связка: `pivot()`/`pivot_table()` + `aggfunc()`

In [ ]:
fix_info_pivot = fix_info.pivot_table(
    index="car_id",  # Строка, для которой хотим сгенерировать признаки
    columns=["work_type"],  # колонка, которую вытяним в столбец
    values=["destroy_degree"],  # столбец, по которой будем считать статистики
    aggfunc=["mean", 'count'],  # признаки для генерации (mean/count/sum)
).fillna(0)

fix_info_pivot.columns = [f"{i[2]}_{i[0]}" for i in fix_info_pivot.columns]
fix_info_pivot.reset_index(inplace=True)

fix_info_pivot.sample(3)

## Рандомы

In [ ]:
np.r_[1:9] # от 1 до 9
np.arange(0,np.pi,0.1) # от нуля до пи с шагом 0.1
np.cos(np.arange(0,np.pi,0.1))
np.linspace(1,5,10) # равномерное разделение
np.logspace(2,5,4) # 10 в степени нач., 10 в степени конечной, количество
np.geomspace(1,20,3) # начальное, конечное, количество. Геометрическая прогрессия

np.random.choice(1000,size=1000, replace =False)
np.random.choice(<из_списка>,size=(3,2))
np.random.randint(<максимальное>,size = (3,2))
np.random.random(10) # кол-во =10, значения от 0 до 1
a,b,c = np.random.normal(0,1,size=(3,100))

# Train Test split

## Train + Test + Validation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y) #stratify принимает только одномерный массив

In [ ]:
from sklearn.model_selection import train_test_split

# Разделение на тренировочные и временные (тест + валидация) данные
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
# Разделим временный на тестовые и валидационные данные
test_df, val_df = train_test_split(temp_df, test_size=0.3, random_state=42)  

# Вывод размеров полученных наборов данных
print(f"Train set: {train_df.shape}")
print(f"Test set: {test_df.shape}")
print(f"Validation set: {val_df.shape}")

y_train, y_valid, y_test = train_df['target'], val_df['target'], test_df['target']
X_train, X_valid, X_test = train_df[all_features], val_df[all_features], test_df[all_features]

## Train (undersampling) + Test

In [ ]:
# Разделение на тренировочный и тестовый наборы данных
train, test = train_test_split(df_lc, test_size=0.2, random_state=42)

# Отбор данных класса 1
class_1 = train[train['target'] == 1]
# Отбор данных класса 0
class_0 = train[train['target'] == 0].sample(n=len(class_1)*2, random_state=42)
# Объединение выборок
balanced_train = pd.concat([class_1, class_0])

X_train_b = balanced_train[all_features]
y_train_b = balanced_train['target']

X_test_b = test[all_features]
y_test_b = test['target']

In [ ]:
categorical_features = list(df_lc.select_dtypes(include=['object']).columns)
numeric_features = list(set(df_lc.columns) - (set(categorical_features) | set(['target'])))
all_features = categorical_features + numeric_features